In [ ]:
DATA_PATH = 'data/no_exogenous/data.csv'
FREQ = 'h'
YEAR = 2021
OUTPUT_PATH = f'data/predictions_prophet_{YEAR}.csv'
HOLIDAYS = ["01-01","04-21","05-01","09-07","10-12","11-02","11-15","11-20","12-25"]

In [ ]:
import pandas as pd
from prophet import Prophet

In [ ]:
data = pd.read_csv(DATA_PATH, parse_dates=['timestamp'])
data = data.set_index('timestamp').asfreq(FREQ).dropna()
data['year'] = data.index.year

train = data[(data['year'] < YEAR) & (data['year'] > YEAR - 5)]
test = data[data['year'] == YEAR]

h = pd.to_datetime(HOLIDAYS, format='%m-%d')
hol = pd.DataFrame({'holiday':'h','ds':h})


In [ ]:
df = train.reset_index()[['timestamp','value']].rename(columns={'timestamp':'ds','value':'y'})
m = Prophet(yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=True,
            seasonality_mode='multiplicative',
            changepoint_prior_scale=0.1,
            seasonality_prior_scale=10,
            holidays_prior_scale=10,
            holidays=hol,
            mcmc_samples=200)
m.add_seasonality(name='monthly', period=30.5, fourier_order=10)
m.fit(df)

In [ ]:
future = m.make_future_dataframe(periods=len(test), freq=FREQ)
fc = m.predict(future)
y_pred = fc['yhat'].iloc[-len(test):].values


In [ ]:
out = test.copy()
out['value'] = y_pred
out.to_csv(OUTPUT_PATH, index=False)